In [ ]:
!pip install tqdm

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import bs4
import numpy as np
from tqdm import tqdm
from IPython.display import display
from tqdm import tqdm


# Accessing the website URL using proxies

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36"
}
# for i in proxies:
#     try:
#         print(f"Using proxy: {i}")
#         res = requests.get("https://www.ambitionbox.com/list-of-companies?page=1", proxies = {"http" : i,  "https" : i}, headers = headers)
#         if res.status_code == 200:
#             print("PROXY SUCCESSFULL.")
#             webpage = res.text
#             break
#     except:
#         print(f"Proxy {i} is not working")
#         continue

In [ ]:
soup = BeautifulSoup(webpage, 'lxml')

In [ ]:
print(soup.prettify())

In [ ]:
soup.find_all('h1')[0].text.strip('\n\t')

In [ ]:
soup.find_all('h2')

In [ ]:
Company_name=[]
Rating = []
Industry=[]
Reviews=[]
URLs=[]
Highly_rated_for=[]
Critically_rated_for=[]

In [ ]:
for i in soup.find_all('h2')[:-4]:
    Company_name.append(i.text.strip('\n\t'))

Company_name

In [ ]:
for i in soup.find_all('span',class_="companyCardWrapper__companyRatingValue"):
    Rating.append(i.text)

In [ ]:
for i in soup.find_all('span',class_="companyCardWrapper__interLinking"):
    Industry.append(i.text.strip('\n\t').split('|')[0].strip())

In [ ]:
Industry

In [ ]:
for i in soup.find_all('meta', itemprop="url"):
    URLs.append(i.attrs['content'])

In [ ]:
URLs = URLs[1:]

In [ ]:
URLs

In [ ]:

for element in soup.find_all('div',class_="companyCardWrapper"):
    tag = element.find('div', attrs = {'class' : "companyCardWrapper__ratingComparisonWrapper" })

    if tag:
        children_tags = [child for child in tag.children if isinstance(child, bs4.element.Tag) and child.name == "div"]
        if (len(children_tags) == 1):
            tag_obj = children_tags[0]
            if(tag_obj.find('span',attrs = {'class' : "companyCardWrapper__ratingHeader--high" })):
                Highly_rated_for.append(tag_obj.find('span', attrs={'class' : "companyCardWrapper__ratingValues"}).text.strip())
                Critically_rated_for.append(np.nan)
            else:
                Highly_rated_for.append(np.nan)
                Critically_rated_for.append(tag_obj.find('span', attrs={'class' : "companyCardWrapper__ratingValues"}).text.strip())

        else:
            Highly_rated_for.append(children_tags[0].find('span', attrs={'class' : "companyCardWrapper__ratingValues"}).text.strip())
            Critically_rated_for.append(children_tags[1].find('span', attrs = {'class' : "companyCardWrapper__ratingValues"}).text.strip())

    else:
        continue

In [ ]:
print(Highly_rated_for)
print(Critically_rated_for)

In [ ]:
for i in soup.find_all('a',href=True,class_="companyCardWrapper__ActionWrapper"):
    href = i.get('href')
    if 'reviews' in href.lower():
        Reviews.append(i.find('span',attrs={'class':"companyCardWrapper__ActionCount"}).text.strip())

In [ ]:
Reviews

In [ ]:
df = pd.DataFrame({'Company_name': Company_name,'Ratings': Rating,'Industry' : Industry,'Reviews' : Reviews,'URL' : URLs,'Highly_rated_for' : Highly_rated_for,'Critically_rated_for' : Critically_rated_for})
df

In [ ]:
import queue
import threading
Company_name=[]
Rating = []
Industry=[]
Reviews=[]
Highly_rated_for=[]
Critically_rated_for=[]
Age = []
Description = []    
Headquarters = []
Ownership = []
Employee_count = []
with open('valid_proxies.txt', "r") as f:
    proxies=tuple(f.read().split("\n"))
    
def progress_update(pbar):
    # Update progress bar based on counter value (implementation depends on library)
    pbar.update(1)

def worker(proxies, url_queue, pbar, url_lock):
    while not url_queue.empty():
        url = url_queue.get()
        page_content = download_page(url, proxies)  
        with url_lock:
            if page_content:
                scrap_page(page_content)
                progress_update(pbar)

                
def download_page(url, proxies):
    for proxy in proxies:
        try:
            res = requests.get(url, proxies={"http": proxy, "https": proxy}, headers=headers)
            if res.status_code == 200:
                return res.text
        except:
            print("Proxy is not working.")
            continue
    return None
            
def scrap_page(webpage):
    try:
        soup = BeautifulSoup(webpage, 'lxml')
        for element in soup.find_all('div',class_="companyCardWrapper"):

            for i in element.find('h2',attrs={'class':"companyCardWrapper__companyName"}):
                Company_name.append(i.text.strip('\n\t'))

            rating_tag = element.find('span',attrs={'class' : "companyCardWrapper__companyRatingValue"})
            if rating_tag:
                Rating.append(rating_tag.text.strip())
            else:
                Rating.append(np.nan)

            for i in element.find_all('span',class_="companyCardWrapper__interLinking"):
                Industry.append(i.text.strip('\n\t').split('|')[0].strip())

            for i in element.find_all('meta', attrs = {'itemprop': "url"}):
                url=i.attrs['content']
                webpage = download_page(url, proxies)
                if webpage:
                    scrap_company_page(webpage)
                else:
                    Age.append(np.nan)
                    Description.append(np.nan)
                    Headquarters.append(np.nan)
                    Ownership.append(np.nan)
                    Employee_count.append(np.nan)

            tag = element.find('div', attrs = {'class' : "companyCardWrapper__ratingComparisonWrapper" })
            if tag:
                children_tags = [child for child in tag.children if isinstance(child, bs4.element.Tag) and child.name == "div"]
                if (len(children_tags) == 1):
                    tag_obj = children_tags[0]
                    if(tag_obj.find('span',attrs = {'class' : "companyCardWrapper__ratingHeader--high" })):
                        Highly_rated_for.append(tag_obj.find('span', attrs={'class' : "companyCardWrapper__ratingValues"}).text.strip())
                        Critically_rated_for.append(np.nan)
                    else:
                        Highly_rated_for.append(np.nan)
                        Critically_rated_for.append(tag_obj.find('span', attrs={'class' : "companyCardWrapper__ratingValues"}).text.strip())
                else:
                        Highly_rated_for.append(children_tags[0].find('span', attrs={'class' : "companyCardWrapper__ratingValues"}).text.strip())
                        Critically_rated_for.append(children_tags[1].find('span', attrs = {'class' : "companyCardWrapper__ratingValues"}).text.strip())

            else:
                Highly_rated_for.append(np.nan)
                Critically_rated_for.append(np.nan)
                continue

        for i in soup.find_all('a',href=True,class_="companyCardWrapper__ActionWrapper"):
            href = i.get('href').lower()
            if 'reviews' in href:
                Reviews.append(i.find('span',class_="companyCardWrapper__ActionCount").text.strip())
            else:
                pass

    except AttributeError as e:
        print(f"Error parsing data on page {i}: {e}")
    except Exception as e:
        print(f"Unexpected error on page {i}: {e}")
        
def scrap_company_page(webpage):
    soup = BeautifulSoup(webpage,'lxml')
    table = soup.find_all('ul',class_="aboutTable")
    if table:
        for element in table:
            child_points = [child for child in element.children if child.name=='li']
            age_tag=child_points[0].find(class_="textItem__val aboutItem__value")
            owner_tag= child_points[1].find(class_="textItem__val aboutItem__value")
            employee_tag= child_points[3].find(class_="textItem__val aboutItem__value")
            headquarters_tag= child_points[4].find(class_="textItem__val aboutItem__value")
            Age.append(age_tag.text.strip() if age_tag else np.nan)
            Ownership.append(owner_tag.text.strip() if owner_tag else np.nan)
            Employee_count.append(employee_tag.text.strip() if employee_tag else np.nan)
            Headquarters.append(headquarters_tag.text.strip() if headquarters_tag else np.nan)
    else: 
        Age.append(np.nan)
        Headquarters.append(np.nan)
        Ownership.append(np.nan)
        Employee_count.append(np.nan)
    if(soup.find('div',itemprop="description")):
        tag = soup.find('div',itemprop="description")
        Description.append(tag.text.strip())
    else:
        Description.append(np.nan)

def main():
    url_queue = queue.Queue()
    url_lock = threading.Lock()
    urls = [f"https://www.ambitionbox.com/list-of-companies?page={i}" for i in  range(1, 334)]
    for i in urls:
        url_queue.put(i)
    pbar = tqdm(total=len(urls), desc="Scraping Pages: ")

    threads = []
    for _ in range(4):
        thread = threading.Thread(target=worker, args=(proxies, url_queue, pbar, url_lock))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    print("Scraping completed!")
if __name__ == "__main__":
    main()

Scraping Pages:   5%|▌         | 17/333 [24:35<7:28:15, 85.11s/it] 

Proxy is not working.


Scraping Pages:  43%|████▎     | 142/333 [3:08:13<4:05:49, 77.22s/it]

Proxy is not working.
Proxy is not working.
Proxy is not working.
Proxy is not working.


Scraping Pages:  45%|████▌     | 151/333 [3:20:01<3:59:46, 79.04s/it]

...

Proxy is not working.
Proxy is not working.
Proxy is not working.
Proxy is not working.


Scraping Pages:  54%|█████▍    | 179/333 [3:59:12<3:39:55, 85.69s/it]

Proxy is not working.
Proxy is not working.
Proxy is not working.
Proxy is not working.
Proxy is not working.


Scraping Pages:  99%|█████████▉| 330/333 [7:19:27<03:59, 79.90s/it]  

Scraping completed!


In [14]:
df = pd.DataFrame({'Company_name': Company_name,'Ratings': Rating,'Industry' : Industry,'Review_count' : Reviews,'Highly_rated_for' : Highly_rated_for,'Critically_rated_for'
: Critically_rated_for})

df.to_csv('Company_Details_part1.csv')

In [15]:

print(len(Company_name))
print(len(Rating))
print(len(Industry))
print(len(Highly_rated_for))
print(len(Critically_rated_for))
print(len(Age))
print(len(Description))
print(len(Headquarters))
print(len(Ownership))
print(len(Employee_count))



df2 = pd.DataFrame({'Company_name': Company_name, 'Headquarters' : Headquarters, 'Industry' : Industry, 'Description' : Description, 'Age':Age, 'Ownership' : Ownership, 'Employee_count' : Employee_count,'Ratings': Rating,'Highly_rated_for' : Highly_rated_for,'Critically_rated_for' : Critically_rated_for, 'Review_count' : Reviews})

df2.to_csv('Company_Details.csv')

6600
6600
6600
6600
6600
6600
6600
6600
6600
6600


In [ ]:
print(len(Company_name))
print(len(Rating))
print(len(Industry))
print(len(Reviews))
print(len(URLs))
print(len(Highly_rated_for))
print(len(Critically_rated_for))
# for i in Company_name:
#     print(i+ "\n")

In [16]:
df2

,Company_name,Headquarters,Industry,Description,Age,Ownership,Employee_count,Ratings,Highly_rated_for,Critically_rated_for,Review_count
0,ICICI Prudential Life Insurance,"Mumbai, Maharashtra, India",Insurance,ICICI Prudential Life Insurance Company Limite...,2000 (24 yrs old),Public,10k-50k,4.1,"Job Security, Skill Development / Learning, Sa...",NaN,5.8k
1,Bajaj Finance,"Pune, Maharashtra, India",NBFC,Bajaj Finance Limited is the most diversified ...,1987 (37 yrs old),Private,10k-50k,4.1,"Salary & Benefits, Skill Development / Learnin...",NaN,5.8k
2,Jana Small Finance Bank,"Bangalore/Bengaluru, Karnataka, India",Banking,Jana Small Finance Bank is a Scheduled Commerc...,2006 (18 yrs old),Public,NaN,3.9,NaN,Promotions / Appraisal,5.7k
3,Lupin,"Mumbai, Maharashtra, India",Pharma,Lupin is a global pharmaceutical company offer...,1968 (56 yrs old),Public,10k-50k,4.1,"Skill Development / Learning, Company Culture,...",NaN,5.6k
4,Indian Army,"New Delhi, Delhi, India",Defence & Aerospace,The principal task of the Ministry is to frame...,1895 (129 yrs old),Public,NaN,4.7,"Job Security, Company Culture, Work Satisfaction",NaN,5.2k
...,...,...,...,...,...,...,...,...,...,...,...
6595,Scholastic,"Gurgaon/Gurugram, Haryana, India",Printing & Publishing,Scholastic Corporation is the world’s largest ...,1997 (27 yrs old),Private,5k-10k,4.0,Work Life Balance,"Promotions / Appraisal, Salary & Benefits, Wor...",114
6596,Shri Mahant Indiresh Hospital,"Dehradun, India",Healthcare,The associated Shri Mahant Indiresh hospital i...,2002 (22 yrs old),Private,NaN,3.8,Work Satisfaction,"Promotions / Appraisal, Salary & Benefits",114
6597,RS Software,"Milpitas, United States (USA)",IT Services & Consulting,"RS Software builds global, national and enterp...",1991 (33 yrs old),Public,501-1k,3.6,NaN,"Job Security, Work Life Balance, Promotions / ...",114
6598,Cheers Interactive,"Utrecht, Albert Heijn, Netherlands",Management Consulting,FutureBridge tracks and advises on the future ...,2000 (24 yrs old),Private,501-1k,3.7,NaN,"Promotions / Appraisal, Company Culture, Work ...",114


In [ ]:
df2.isnull().sum()

In [10]:
df2 = pd.read_csv('Company_Details.csv')
df2

,Company_name,Headquarters,Industry,Description,Age,Ownership,Employee_count,Ratings,Highly_rated_for,Critically_rated_for,Review_count
0,ICICI Prudential Life Insurance,"Mumbai, Maharashtra, India",Insurance,ICICI Prudential Life Insurance Company Limite...,2000 (24 yrs old),Public,10k-50k,4.1,"Job Security, Skill Development / Learning, Sa...",NaN,5.8k
1,Bajaj Finance,"Pune, Maharashtra, India",NBFC,Bajaj Finance Limited is the most diversified ...,1987 (37 yrs old),Private,10k-50k,4.1,"Salary & Benefits, Skill Development / Learnin...",NaN,5.8k
2,Jana Small Finance Bank,"Bangalore/Bengaluru, Karnataka, India",Banking,Jana Small Finance Bank is a Scheduled Commerc...,2006 (18 yrs old),Public,NaN,3.9,NaN,Promotions / Appraisal,5.7k
3,Lupin,"Mumbai, Maharashtra, India",Pharma,Lupin is a global pharmaceutical company offer...,1968 (56 yrs old),Public,10k-50k,4.1,"Skill Development / Learning, Company Culture,...",NaN,5.6k
4,Indian Army,"New Delhi, Delhi, India",Defence & Aerospace,The principal task of the Ministry is to frame...,1895 (129 yrs old),Public,NaN,4.7,"Job Security, Company Culture, Work Satisfaction",NaN,5.2k
...,...,...,...,...,...,...,...,...,...,...,...
6595,Scholastic,"Gurgaon/Gurugram, Haryana, India",Printing & Publishing,Scholastic Corporation is the world’s largest ...,1997 (27 yrs old),Private,5k-10k,4.0,Work Life Balance,"Promotions / Appraisal, Salary & Benefits, Wor...",114
6596,Shri Mahant Indiresh Hospital,"Dehradun, India",Healthcare,The associated Shri Mahant Indiresh hospital i...,2002 (22 yrs old),Private,NaN,3.8,Work Satisfaction,"Promotions / Appraisal, Salary & Benefits",114
6597,RS Software,"Milpitas, United States (USA)",IT Services & Consulting,"RS Software builds global, national and enterp...",1991 (33 yrs old),Public,501-1k,3.6,NaN,"Job Security, Work Life Balance, Promotions / ...",114
6598,Cheers Interactive,"Utrecht, Albert Heijn, Netherlands",Management Consulting,FutureBridge tracks and advises on the future ...,2000 (24 yrs old),Private,501-1k,3.7,NaN,"Promotions / Appraisal, Company Culture, Work ...",114
